In [12]:
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os

In [8]:
import cv2
from google.colab.patches import cv2_imshow
import os
# Open the test images file
test_dir = '/content/drive/MyDrive/Image_detection_system/Data/test_SVM'
test_images = []
count_tampered = 0  # initialize a counter variable for tampered images

for file in os.listdir(test_dir):
  if file.endswith('.jpg'):
    image_path = os.path.join(test_dir, file)
    #print(image_path)
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    dim = (384, 256)
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_LANCZOS4)  # resize image
    test_images.append(resized)
    count_tampered += 1

### FUNCTIONAL CODE for the test images

In [13]:
def preprocess(img):
    # Convert the image to YCbCr color space
    img_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    # Extract the Cr channel
    img_y, img_cr, img_cb = cv2.split(img_ycrcb)
    # Convert the channels to floating-point format
    img_y = np.float32(img_y)
    img_cr = np.float32(img_cr)
    img_cb = np.float32(img_cb)
    # Apply non-overlapping 8x8 block-wise DCT to each channel
    img_y_dct = np.zeros_like(img_y)
    img_cr_dct = np.zeros_like(img_cr)
    img_cb_dct = np.zeros_like(img_cb)
    for i in range(0, img_y.shape[0], 8):
        for j in range(0, img_y.shape[1], 8):
            img_y_dct[i:i+8, j:j+8] = cv2.dct(img_y[i:i+8, j:j+8])
            img_cr_dct[i:i+8, j:j+8] = cv2.dct(img_cr[i:i+8, j:j+8])
            img_cb_dct[i:i+8, j:j+8] = cv2.dct(img_cb[i:i+8, j:j+8])
    # Round DCT coefficients to nearest integer values for each channel
    img_y_dct = np.round(img_y_dct)
    img_cr_dct = np.round(img_cr_dct)
    img_cb_dct = np.round(img_cb_dct)
    # Calculate decorrelation features for each channel
    Dhintra_y = img_y_dct[:-1, :] - img_y_dct[1:, :]
    Dvintra_y = img_y_dct[:, :-1] - img_y_dct[:, 1:]
    Dhinter_y = img_y_dct[:-8, :] - img_y_dct[8:, :]
    Dvinter_y = img_y_dct[:, :-8] - img_y_dct[:, 8:]

    Dhintra_cr = img_cr_dct[:-1, :] - img_cr_dct[1:, :]
    Dvintra_cr = img_cr_dct[:, :-1] - img_cr_dct[:, 1:]
    Dhinter_cr = img_cr_dct[:-8, :] - img_cr_dct[8:, :]
    Dvinter_cr = img_cr_dct[:, :-8] - img_cr_dct[:, 8:]

    Dhintra_cb = img_cb_dct[:-1, :] - img_cb_dct[1:, :]
    Dvintra_cb = img_cb_dct[:, :-1] - img_cb_dct[:, 1:]
    Dhinter_cb = img_cb_dct[:-8, :] - img_cb_dct[8:, :]
    Dvinter_cb = img_cb_dct[:, :-8] - img_cb_dct[:, 8:]

    return Dhintra_y, Dvintra_y, Dhinter_y, Dvinter_y, Dhintra_cr, Dvintra_cr, Dhinter_cr, Dvinter_cr, Dhintra_cb, Dvintra_cb, Dhinter_cb, Dvinter_cb

In [14]:
#Function to apply the Markov random process
def apply_markov_process(Dh_intra, Dv_intra, Dh_inter, Dv_inter, threshold=8):
    # Resize the matrices to the same size
    min_rows = min(Dh_intra.shape[0], Dv_intra.shape[0], Dh_inter.shape[0], Dv_inter.shape[0])
    min_cols = min(Dh_intra.shape[1], Dv_intra.shape[1], Dh_inter.shape[1], Dv_inter.shape[1])
    Dh_intra = Dh_intra[:min_rows, :min_cols]
    Dv_intra = Dv_intra[:min_rows, :min_cols]
    Dh_inter = Dh_inter[:min_rows, :min_cols]
    Dv_inter = Dv_inter[:min_rows, :min_cols]
    # Apply the Markov random process
    markov_matrix = np.zeros(Dh_intra.shape)
    for i in range(1, Dh_intra.shape[0]):
        for j in range(1, Dh_intra.shape[1]):
            if abs(Dh_intra[i, j]) > threshold or abs(Dv_intra[i, j]) > threshold or abs(Dh_inter[i, j]) > threshold or abs(Dv_inter[i, j]) > threshold:
                markov_matrix[i, j] = 1
    return markov_matrix

In [22]:
# Extract features using texture descriptors (e.g. LBP) and color histograms
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray

def extract_features(image):
    gray = rgb2gray(image)
    lbp = local_binary_pattern(gray, 8, 1)
    hist, _ = np.histogram(lbp.ravel(), bins=256)
    color_hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    features = np.concatenate((hist, color_hist.flatten()))
    return features

### DRIVER CODE

In [16]:
test_processed = []
for img in test_images:
    if img is not None and img.size > 0:
        img = img.astype(np.float32) # Convert the image to 32-bit float format
        Dhintra_y, Dvintra_y, Dhinter_y, Dvinter_y, Dhintra_cr, Dvintra_cr, Dhinter_cr, Dvinter_cr, Dhintra_cb, Dvintra_cb, Dhinter_cb, Dvinter_cb = preprocess(img)
        markov_matrix1 = apply_markov_process(Dhintra_y, Dvintra_y, Dhinter_y, Dvinter_y)
        markov_matrix2 = apply_markov_process(Dhintra_cr, Dvintra_cr, Dhinter_cr, Dvinter_cr)
        markov_matrix3 = apply_markov_process( Dhintra_cb, Dvintra_cb, Dhinter_cb, Dvinter_cb)
        feature1 = markov_matrix1.flatten()
        feature2 = markov_matrix2.flatten()
        feature3 = markov_matrix3.flatten()
        features = np.concatenate((feature1, feature2, feature3), axis=0)
        test_processed.append(features)
    else:
        print("Invalid image encountered in training set, skipping...")


In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import joblib

In [33]:
#get the path or directory
test_folder_dir = '/content/drive/MyDrive/Image_detection_system/Data/test_SVM'
print("Labels are 0 for non-tampered and 1 for tampered")

count_images, count_authentic, count_tampered = 0, 0, 0
test_images = []
num_samples = 20
filename = '/content/drive/MyDrive/Image_detection_system/Environments/ML_implementation/ML_svm_model_traindev.joblib'
svm_model = joblib.load(filename)
svm_model_class = joblib.load('/content/drive/MyDrive/Image_detection_system/Environments/ML_implementation/svm_model_traindev.pkl')

for file in os.listdir(test_folder_dir):

    # check if the image ends with png or jpg or jpeg
    if (file.endswith(".png") or file.endswith(".jpg")\
        or file.endswith(".jpeg") or file.endswith(".tif")):
        image_path = os.path.join(test_dir, file)
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        dim = (384, 256)
        resized = cv2.resize(image, dim, interpolation = cv2.INTER_LANCZOS4)  # resize image
        test_images.append(resized)
        count_images += 1
        if count_images == num_samples:
          break

        test_processed = []
        for img in test_images:
          if img is not None and img.size > 0:
            img = img.astype(np.float32) # Convert the image to 32-bit float format
            Dhintra_y, Dvintra_y, Dhinter_y, Dvinter_y, Dhintra_cr, Dvintra_cr, Dhinter_cr, Dvinter_cr, Dhintra_cb, Dvintra_cb, Dhinter_cb, Dvinter_cb = preprocess(img)
            markov_matrix1 = apply_markov_process(Dhintra_y, Dvintra_y, Dhinter_y, Dvinter_y)
            markov_matrix2 = apply_markov_process(Dhintra_cr, Dvintra_cr, Dhinter_cr, Dvinter_cr)
            markov_matrix3 = apply_markov_process( Dhintra_cb, Dvintra_cb, Dhinter_cb, Dvinter_cb)
            feature1 = markov_matrix1.flatten()
            feature2 = markov_matrix2.flatten()
            feature3 = markov_matrix3.flatten()
            features = np.concatenate((feature1, feature2, feature3), axis=0)
            test_processed.append(features)
          else:
            print("Invalid image encountered in training set, skipping...")

        test_processed_reshaped = np.array(test_processed, dtype=object)
        prediction_svm = svm_model.predict(test_processed_reshaped)
        print("Image name: ", file)
        print("Tampered prediction: ", prediction_svm[0])
        if(prediction_svm[0] == 1):
          count_tampered += 1
          new_features = [extract_features(image) for image in test_images]
          new_prediction = svm_model_class.predict(new_features)
          print(classification_report(prediction_svm,new_prediction)) # print the classification report
          if new_prediction[-1] == 1:
            print('The image is copy-moved.')
          else:
            print('The image is spliced.')
        else:
          count_authentic += 1

print("No of authentic images: ", count_authentic)
print("No of tampered images: ", count_tampered)

Labels are 0 for non-tampered and 1 for tampered
Image name:  c_0011.tif
Tampered prediction:  1
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

The image is copy-moved.
Image name:  c_0025.tif
Tampered prediction:  1
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

The image is copy-moved.
Image name:  c_0024.jpg
Tampered prediction:  1
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Image name:  c_0018.tif
Tampered prediction:  1
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      0.67      0.67         3

    accuracy                           0.50         4
   macro avg       0.33      0.33      0.33         4
weighted avg       0.50      0.50      0.50         4

The image is spliced.
Image name:  c_0021.tif
Tampered prediction:  1
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.50      0.67      0.57         3

    accuracy                           0.40         5
   macro avg       0.25      0.33      0.29         5
weighted avg       0.30      0.40      0.34         5

The image is copy-moved.
Image name:  c_0017.jpg
Tampered prediction:  1
              precision    recall  f1-score   support

           0       0.50      0.33      0.40         3
           1       0.50      0.67      0.57   